Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---


Logistic model with L2 regularization:


In [5]:
batch_size = 128
beta = 0.001

graph = tf.Graph()
with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables    
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) 
        + beta*tf.nn.l2_loss(weights))
        
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 15.539800
Minibatch accuracy: 14.1%
Validation accuracy: 19.6%
Minibatch loss at step 500: 2.440537
Minibatch accuracy: 78.1%
Validation accuracy: 77.5%
Minibatch loss at step 1000: 1.754835
Minibatch accuracy: 78.1%
Validation accuracy: 78.6%
Minibatch loss at step 1500: 1.294792
Minibatch accuracy: 78.1%
Validation accuracy: 80.6%
Minibatch loss at step 2000: 0.860437
Minibatch accuracy: 81.2%
Validation accuracy: 81.2%
Minibatch loss at step 2500: 0.745235
Minibatch accuracy: 85.2%
Validation accuracy: 82.3%
Minibatch loss at step 3000: 0.840968
Minibatch accuracy: 75.8%
Validation accuracy: 81.8%
Test accuracy: 88.2%


Neural Network model with L2 regularization:

In [7]:
batch_size = 128
beta = 0.001
n_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(tf.truncated_normal([n_hidden_nodes, num_labels])) 
    biases = tf.Variable(tf.zeros([num_labels]))
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden_nodes]) )
    hidden_biases = tf.Variable(tf.zeros([n_hidden_nodes]))
    hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
  
    # Training computation.  
    logits = tf.matmul(hidden_layer, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
        + beta * tf.nn.l2_loss(hidden_weights)
        + tf.nn.l2_loss(weights))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    valid_prediction = tf.nn.softmax( tf.matmul(valid_relu, weights) + biases) 
    test_relu = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
    test_prediction = tf.nn.softmax(tf.matmul(test_relu, weights) + biases)

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4521.123535
Minibatch accuracy: 10.9%
Validation accuracy: 25.6%
Minibatch loss at step 500: 193.736389
Minibatch accuracy: 57.0%
Validation accuracy: 49.2%
Minibatch loss at step 1000: 113.035065
Minibatch accuracy: 68.8%
Validation accuracy: 72.6%
Minibatch loss at step 1500: 69.282669
Minibatch accuracy: 66.4%
Validation accuracy: 79.0%
Minibatch loss at step 2000: 41.339046
Minibatch accuracy: 75.8%
Validation accuracy: 77.0%
Minibatch loss at step 2500: 25.574760
Minibatch accuracy: 78.1%
Validation accuracy: 77.9%
Minibatch loss at step 3000: 16.097744
Minibatch accuracy: 71.1%
Validation accuracy: 75.0%
Test accuracy: 80.9%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [9]:
batch_size = 128
beta = 0.001
n_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(tf.truncated_normal([n_hidden_nodes, num_labels])) 
    biases = tf.Variable(tf.zeros([num_labels]))
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden_nodes]) )
    hidden_biases = tf.Variable(tf.zeros([n_hidden_nodes]))
    hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
  
    # Training computation.  
    logits = tf.matmul(hidden_layer, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
        + beta * tf.nn.l2_loss(hidden_weights)
        + tf.nn.l2_loss(weights))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    valid_prediction = tf.nn.softmax( tf.matmul(valid_relu, weights) + biases) 
    test_relu = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
    test_prediction = tf.nn.softmax(tf.matmul(test_relu, weights) + biases)

In [10]:
num_steps = 101
num_batches = 2

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = step % num_batches
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 10 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4794.396484
Minibatch accuracy: 9.4%
Validation accuracy: 20.3%
Minibatch loss at step 10: 459.511353
Minibatch accuracy: 31.2%
Validation accuracy: 16.6%
Minibatch loss at step 20: 384.196625
Minibatch accuracy: 56.2%
Validation accuracy: 44.9%
Minibatch loss at step 30: 433.372711
Minibatch accuracy: 82.0%
Validation accuracy: 33.7%
Minibatch loss at step 40: 348.153870
Minibatch accuracy: 49.2%
Validation accuracy: 37.7%
Minibatch loss at step 50: 337.369080
Minibatch accuracy: 64.8%
Validation accuracy: 20.9%
Minibatch loss at step 60: 307.180725
Minibatch accuracy: 83.6%
Validation accuracy: 12.2%
Minibatch loss at step 70: 303.866150
Minibatch accuracy: 87.5%
Validation accuracy: 27.9%
Minibatch loss at step 80: 405.671967
Minibatch accuracy: 35.9%
Validation accuracy: 50.1%
Minibatch loss at step 90: 311.674713
Minibatch accuracy: 51.6%
Validation accuracy: 19.3%
Minibatch loss at step 100: 311.292938
Minibatch accuracy: 69.5%
Validation acc

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [11]:
batch_size = 128
beta = 0.01
n_hidden_nodes = 1024
keep_probability = 0.7

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(tf.truncated_normal([n_hidden_nodes, num_labels])) 
    biases = tf.Variable(tf.zeros([num_labels]))
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden_nodes]) )
    hidden_biases = tf.Variable(tf.zeros([n_hidden_nodes]))
    hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
  
    # Training computation.
    dropout = tf.nn.dropout(hidden_layer, keep_probability) 
    logits = tf.matmul(dropout, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
        + beta * tf.nn.l2_loss(hidden_weights)
        + tf.nn.l2_loss(weights))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    valid_prediction = tf.nn.softmax( tf.matmul(valid_relu, weights) + biases) 
    test_relu = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
    test_prediction = tf.nn.softmax(tf.matmul(test_relu, weights) + biases)

In [12]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 7369.501465
Minibatch accuracy: 8.6%
Validation accuracy: 28.1%
Minibatch loss at step 500: 21.880751
Minibatch accuracy: 74.2%
Validation accuracy: 77.5%
Minibatch loss at step 1000: 1.775488
Minibatch accuracy: 72.7%
Validation accuracy: 80.2%
Minibatch loss at step 1500: 2.162793
Minibatch accuracy: 61.7%
Validation accuracy: 74.7%
Minibatch loss at step 2000: 2.169295
Minibatch accuracy: 62.5%
Validation accuracy: 65.2%
Minibatch loss at step 2500: 1.819752
Minibatch accuracy: 68.8%
Validation accuracy: 69.2%
Minibatch loss at step 3000: 1.710865
Minibatch accuracy: 71.9%
Validation accuracy: 76.4%
Test accuracy: 82.5%


In [13]:
# now for the overfitting case

batch_size = 128
beta = 0.001
n_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(tf.truncated_normal([n_hidden_nodes, num_labels])) 
    biases = tf.Variable(tf.zeros([num_labels]))
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden_nodes]) )
    hidden_biases = tf.Variable(tf.zeros([n_hidden_nodes]))
    hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
  
    # Training computation.  
    logits = tf.matmul(hidden_layer, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
        + beta * tf.nn.l2_loss(hidden_weights)
        + tf.nn.l2_loss(weights))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    valid_prediction = tf.nn.softmax( tf.matmul(valid_relu, weights) + biases) 
    test_relu = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
    test_prediction = tf.nn.softmax(tf.matmul(test_relu, weights) + biases)

In [14]:
num_steps = 101
num_batches = 2

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = step % num_batches
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 10 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4516.330078
Minibatch accuracy: 10.2%
Validation accuracy: 22.7%
Minibatch loss at step 10: 583.048462
Minibatch accuracy: 59.4%
Validation accuracy: 26.0%
Minibatch loss at step 20: 566.009033
Minibatch accuracy: 23.4%
Validation accuracy: 45.4%
Minibatch loss at step 30: 350.244202
Minibatch accuracy: 59.4%
Validation accuracy: 37.6%
Minibatch loss at step 40: 338.366272
Minibatch accuracy: 52.3%
Validation accuracy: 39.1%
Minibatch loss at step 50: 333.154480
Minibatch accuracy: 63.3%
Validation accuracy: 16.8%
Minibatch loss at step 60: 365.789185
Minibatch accuracy: 87.5%
Validation accuracy: 54.4%
Minibatch loss at step 70: 327.686066
Minibatch accuracy: 82.0%
Validation accuracy: 17.9%
Minibatch loss at step 80: 317.220215
Minibatch accuracy: 84.4%
Validation accuracy: 33.8%
Minibatch loss at step 90: 355.575012
Minibatch accuracy: 85.2%
Validation accuracy: 50.4%
Minibatch loss at step 100: 295.742920
Minibatch accuracy: 89.8%
Validation ac

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [15]:
batch_size = 128
n_hidden_nodes1 = 1024
n_hidden_nodes2 = 512
n_hidden_nodes3 = 512
beta = 0.0001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  global_step = tf.Variable(0)

  # Variables.
  weights1 = tf.Variable(tf.truncated_normal(
        [image_size * image_size, n_hidden_nodes1],
        stddev=np.sqrt(2.0 / (image_size * image_size))))
  biases1 = tf.Variable(tf.zeros([n_hidden_nodes1]))
  weights2 = tf.Variable(tf.truncated_normal(
        [n_hidden_nodes1, n_hidden_nodes2], 
        stddev=np.sqrt(2.0 / n_hidden_nodes1)))
  biases2 = tf.Variable(tf.zeros([n_hidden_nodes2]))
  weights3 = tf.Variable(tf.truncated_normal(
        [n_hidden_nodes2, n_hidden_nodes3], 
        stddev=np.sqrt(2.0 / n_hidden_nodes2)))
  biases3 = tf.Variable(tf.zeros([n_hidden_nodes3]))
  weights4 = tf.Variable(tf.truncated_normal(
        [n_hidden_nodes3, num_labels], 
        stddev=np.sqrt(2.0 / n_hidden_nodes3)))
  biases4 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  train1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  train2 = tf.nn.relu(tf.matmul(train1, weights2) + biases2)
  train3 = tf.nn.relu(tf.matmul(train2, weights3) + biases3)
  logits = tf.matmul(train3, weights4) + biases4
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
    + beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3))
  )
  
  # Optimizer.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 4000, 0.65, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid2 = tf.nn.relu(tf.matmul(valid1, weights2) + biases2)
  valid3 = tf.nn.relu(tf.matmul(valid2, weights3) + biases3)
  valid_prediction = tf.nn.softmax(tf.matmul(valid3, weights4) + biases4)
  test1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  test2 = tf.nn.relu(tf.matmul(test1, weights2) + biases2)
  test3 = tf.nn.relu(tf.matmul(test2, weights3) + biases3)
  test_prediction = tf.nn.softmax(tf.matmul(test3, weights4) + biases4)

In [16]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.548082
Minibatch accuracy: 9.4%
Validation accuracy: 16.3%
Minibatch loss at step 1000: 0.618610
Minibatch accuracy: 83.6%
Validation accuracy: 88.5%
Minibatch loss at step 2000: 0.443228
Minibatch accuracy: 92.2%
Validation accuracy: 88.9%
Minibatch loss at step 3000: 0.458529
Minibatch accuracy: 89.8%
Validation accuracy: 89.9%
Minibatch loss at step 4000: 0.464498
Minibatch accuracy: 89.8%
Validation accuracy: 90.2%
Minibatch loss at step 5000: 0.275435
Minibatch accuracy: 95.3%
Validation accuracy: 90.8%
Minibatch loss at step 6000: 0.298631
Minibatch accuracy: 94.5%
Validation accuracy: 90.8%
Minibatch loss at step 7000: 0.391935
Minibatch accuracy: 91.4%
Validation accuracy: 90.8%
Minibatch loss at step 8000: 0.261965
Minibatch accuracy: 96.1%
Validation accuracy: 90.9%
Minibatch loss at step 9000: 0.330631
Minibatch accuracy: 93.0%
Validation accuracy: 91.0%
Minibatch loss at step 10000: 0.296595
Minibatch accuracy: 94.5%
Validation accura